In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
X, y = fetch_california_housing(return_X_y=True,data_home = "./")
x_train,x_val,y_train,y_val = train_test_split(X,y,train_size=0.8)

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [3]:
x_train_tensor = torch.as_tensor(x_train,dtype=torch.float32)
y_train_tensor = torch.as_tensor(y_train,dtype=torch.float32).view(-1,1)
x_test_tensor = torch.as_tensor(x_val,dtype=torch.float32)
y_test_tensor = torch.as_tensor(y_val,dtype=torch.float32).view(-1,1)

In [4]:
train_dataset = TensorDataset(x_train_tensor,y_train_tensor)
test_dataset = TensorDataset(x_test_tensor,y_test_tensor)
train_loader = DataLoader(train_dataset,batch_size=400,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=200,shuffle=False)

In [5]:
class MylinearRegression(nn.Module):
    def __init__ (self):
        super().__init__()
        self.fc1 = nn.Linear(8,64)
        self.fc2 = nn.Linear(64,128)
        self.fc3 = nn.Linear(128,64)
        self.fc4 = nn.Linear(64,1)

    def forward(self,x):
        y = F.relu(self.fc1(x))
        y = F.relu(self.fc2(y))
        y = F.relu(self.fc3(y))
        y = self.fc4(y)
        return y

In [6]:
model = MylinearRegression()

In [7]:
def mini_batch_train(model,train_loader,optimizer,loss_fn):
    mini_batch_losses = []
    for x_batch,y_batch in train_loader:
        model.train()
        yhat = model(x_batch)
        loss = loss_fn(yhat,y_batch)
        mini_batch_losses.append(loss.item())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return np.mean(mini_batch_losses)

In [8]:
def mini_batch_test(model,test_loader,loss_fn):
    mini_batch_losses = []
    for x_batch,y_batch in test_loader:
        model.eval()
        yhat = model(x_batch)
        loss = loss_fn(yhat,y_batch)
        mini_batch_losses.append(loss.item())
    
    return np.mean(mini_batch_losses)   

In [9]:
def train(model,train_loader,test_loader,optimizer,loss_fn):
    np.random.seed(23)
    torch.manual_seed(23)
    train_losses = []
    test_losses = []
    total_epoches = 0
    epochs = 1000
    for epoch in range(epochs):
        model.train()
        train_loss = mini_batch_train(model,train_loader,optimizer,loss_fn)
        train_losses.append(train_loss)
        total_epoches += 1
        with torch.no_grad():
            model.eval()
            test_loss = mini_batch_test(model,test_loader,loss_fn)
            test_losses.append(test_loss)
        
        print(f"Epoch {epoch+1}/{epochs}, train_loss: {train_loss:.4f}, test_loss: {test_loss:.4f}")

    return train_losses,test_losses


In [10]:
learnRate = 0.001
optimizer = optim.Adam(model.parameters(),lr = learnRate)
loss_fn = nn.MSELoss(reduction = 'mean')
train_losses,test_losses = train(model,train_loader,test_loader,optimizer,loss_fn)
print(train_losses,test_losses)
print(model.state_dict())

Epoch 1/1000, train_loss: 27.8739, test_loss: 2.5447
Epoch 2/1000, train_loss: 1.5695, test_loss: 1.5365
Epoch 3/1000, train_loss: 1.3271, test_loss: 1.3352
Epoch 4/1000, train_loss: 1.2623, test_loss: 1.2902
Epoch 5/1000, train_loss: 1.1857, test_loss: 1.2065
Epoch 6/1000, train_loss: 1.1088, test_loss: 1.1166
Epoch 7/1000, train_loss: 1.0335, test_loss: 1.1011
Epoch 8/1000, train_loss: 0.9386, test_loss: 0.9948
Epoch 9/1000, train_loss: 0.8374, test_loss: 0.8367
Epoch 10/1000, train_loss: 0.7436, test_loss: 0.7541
Epoch 11/1000, train_loss: 0.6825, test_loss: 0.6963
Epoch 12/1000, train_loss: 0.7223, test_loss: 0.6942
Epoch 13/1000, train_loss: 0.6316, test_loss: 0.6624
Epoch 14/1000, train_loss: 0.6331, test_loss: 0.7581
Epoch 15/1000, train_loss: 0.6342, test_loss: 0.6441
Epoch 16/1000, train_loss: 0.6262, test_loss: 0.6454
Epoch 17/1000, train_loss: 0.6195, test_loss: 0.6451
Epoch 18/1000, train_loss: 0.5788, test_loss: 0.7326
Epoch 19/1000, train_loss: 0.5822, test_loss: 1.1374
E

KeyboardInterrupt: 